<a href="https://colab.research.google.com/github/ayalarodrigues/LLL-RAG-Chatbot-Suporte/blob/main/LLM_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação das bibliotecas


In [2]:
!pip install langchain langchain-groq langchain_community langchain-huggingface --q
!pip install faiss-cpu sentence-transformers PyMuPDF --q

!pip install -U sentence-transformers
!pip install -U transformers


#langchain-grop: acesso aos modelos via API
#faiss-cpu: onde serão armazenados os documentos(pode ser cpu ou gpu)
#sentence-transformers: conversão dos textos para números
#PyMuPDF: leitura e processamento de texto


Importações

In [3]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
import getpass

Carregamento da LLM

In [4]:
os.environ["GROQ_API_KEY"] = getpass.getpass()
#getpass não exibe a chave da API no CF


··········


In [5]:
def load_llm(id_model, temperature):
  llm = ChatGroq(
      model = id_model,
      temperature = temperature,
      max_tokens = None,
      timeout = None,
      max_retries = 2
  )
  return llm

In [6]:
#teste
id_model = "llama3-70b-8192" # @param {type: "string"}
temperature = 0.7 # @param {type: "slider", min: 0.1, max: 1.5, step: 0.1}

llm = load_llm(id_model, temperature)

In [7]:
#teste
prompt = "Como alterar minha senha?" # @param {type: "string"}

template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente virtual prestativo e está respondendo perguntas gerais"),
    ("human", "{prompt}")
])

chain = template | llm | StrOutputParser()

res = chain.invoke({"prompt": prompt})
res

'Alterar a senha é uma boa prática de segurança!\n\nO processo de alterar a senha varia dependendo do sistema ou aplicativo que você está usando. Aqui estão as etapas gerais para alterar a senha em alguns dos mais comuns serviços e sistemas:\n\n**Conta de e-mail:**\n\n1. Faça login na sua conta de e-mail.\n2. Clique em "Configurações" ou "Opções" e selecione "Segurança" ou "Conta".\n3. Clique em "Alterar senha" ou "Redefinir senha".\n4. Digite a senha atual e a nova senha desejada.\n5. Confirme a nova senha e clique em "Salvar".\n\n**Sites de redes sociais:**\n\n1. Faça login na sua conta de rede social.\n2. Clique em "Configurações" ou "Opções" e selecione "Segurança" ou "Conta".\n3. Clique em "Alterar senha" ou "Redefinir senha".\n4. Digite a senha atual e a nova senha desejada.\n5. Confirme a nova senha e clique em "Salvar".\n\n**Contas bancárias online:**\n\n1. Faça login na sua conta bancária online.\n2. Clique em "Configurações" ou "Opções" e selecione "Segurança" ou "Conta".\n3.

In [8]:
def show_res(res):
  from IPython.display import display, Markdown
  if "</think>" in res:
    res = res.split("</think>")[-1].strip()  #o modelo pode retornar o passo a passo necessário para resolução do problema
  else:
    res = res.strip()
  display(Markdown(res))


show_res(res)

 #a tag </think> serve para delimitar pensamentos intermediários, o que permite visualizar o "raciocínio" utilizado na resposta
 #optei por remover a tag por não ser de interesse da persona

Alterar a senha é uma boa prática de segurança!

O processo de alterar a senha varia dependendo do sistema ou aplicativo que você está usando. Aqui estão as etapas gerais para alterar a senha em alguns dos mais comuns serviços e sistemas:

**Conta de e-mail:**

1. Faça login na sua conta de e-mail.
2. Clique em "Configurações" ou "Opções" e selecione "Segurança" ou "Conta".
3. Clique em "Alterar senha" ou "Redefinir senha".
4. Digite a senha atual e a nova senha desejada.
5. Confirme a nova senha e clique em "Salvar".

**Sites de redes sociais:**

1. Faça login na sua conta de rede social.
2. Clique em "Configurações" ou "Opções" e selecione "Segurança" ou "Conta".
3. Clique em "Alterar senha" ou "Redefinir senha".
4. Digite a senha atual e a nova senha desejada.
5. Confirme a nova senha e clique em "Salvar".

**Contas bancárias online:**

1. Faça login na sua conta bancária online.
2. Clique em "Configurações" ou "Opções" e selecione "Segurança" ou "Conta".
3. Clique em "Alterar senha" ou "Redefinir senha".
4. Digite a senha atual e a nova senha desejada.
5. Confirme a nova senha e clique em "Salvar".

**Computador ou dispositivo:**

1. Clique no ícone do usuário no canto superior direito da tela.
2. Selecione "Opções" ou "Configurações".
3. Clique em "Alterar senha" ou "Redefinir senha".
4. Digite a senha atual e a nova senha desejada.
5. Confirme a nova senha e clique em "Salvar".

**Observações importantes:**

* Certifique-se de que a nova senha seja forte e única, evitando usar senhas fáceis de adivinhar, como nomes, datas de nascimento ou palavras comuns.
* Use uma senha diferente para cada conta ou serviço.
* É recomendável alterar a senha a cada 60 a 90 dias para manter a segurança da sua conta.

Se você tiver alguma dúvida específica sobre como alterar a senha em um determinado serviço ou sistema, sinta-se à vontade para perguntar!

Definição do contexto

Há basicamente duas possibilidades para o exemplo de teste: o algoritmo não consegue retornar com exatidão o caminho exato para recuperar a senha ou a informação foi "inventada"(processo de alucinação).

O modelo quer apenas prever a próxima combinação de palavras que faça sentido. Nem sempre poderá ser entendido que o modelo não sabe essa informação com exatidão. O modelo pode apenas sugerir um caminho comum, como o retornado no exemplo, porém, o algoritmo não possui meios de conhecer o caminho exato sem consultar uma fonte que comprove.

Nesse aspecto, é importante que seja fornecido um contexto que tenha as informações exatas para a LLM e assim a dúvida do usuário pode ser respondida.

Para a obtenção do contexto, nesse estudo de caso, será feito o carregamento do documento que posteriormente será processado pelo pipeline de RAG.



In [9]:
#teste

context = """
Para alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.
Para alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.
"""

prompt = f"""
Como alterar minha senha?

Contexto: {context}
"""

In [10]:
#teste

prompt

"\nComo alterar minha senha?\n\nContexto: \nPara alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.\nPara alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.\n\n"

In [11]:
#teste

res = chain.invoke({"prompt": prompt})
show_res(res)


Para alterar sua senha, há duas opções:

**No aplicativo:**
1. Clique no menu "Minha conta"
2. Selecione "Alterar senha"

**No site:**
1. Acesse o menu do topo e clique em "Configurações"
2. Selecione "Minha conta"
3. Clique em "Alterar senha"

Siga esses passos e você poderá alterar sua senha com facilidade!

Prompt para o RAG

In [12]:
template_rag = """
Pergunta: {input}
Contexto: {context}
"""

In [13]:
from langchain.prompts import PromptTemplate #é utilizado para prompts personalizados

prompt_rag = PromptTemplate.from_template(template_rag)
print(prompt_rag)


input_variables=['context', 'input'] input_types={} partial_variables={} template='\nPergunta: {input}\nContexto: {context}\n'


Criação dos textos e geração da Chain

In [14]:
chaing_rag = prompt_rag | llm | StrOutputParser()

input = "Como alterar minha senha?"
res = chaing_rag.invoke({"input": input, "context": context}) # o envio de variáveis deve seguir de acordo com o que foi definido no template do prompt
show_res(res)

Para alterar sua senha, siga os seguintes passos:

**No aplicativo:**

1. Clique no menu "Minha conta".
2. Selecione "Alterar senha".

**No site:**

1. Acesse "Configurações" no menu do topo.
2. Selecione "Minha conta".
3. Clique em "Alterar senha".

Siguiendo esses passos, você poderá alterar sua senha com facilidade!

Etapas de Indexação

Para este estudo de caso será feito o uso de funções(Document Loaders) para importar dados na LangChain.

Os documentos serão quebrados para melhor indexação e processamento do modelo.

Na sequência, será feito o embedding(conversão dos textos em representações numéricas) em cada pedaço e por fim os mesmos serão salvos em um banco de dados vetorial.

Carregamento do conteúdo dos documentos

In [15]:
from google.colab import files

uploaded = files.upload()
file_path =list(uploaded.keys())[0]

print(f"Arquivo carregado: {file_path}")

Saving manual_facesecure.pdf to manual_facesecure (1).pdf
Arquivo carregado: manual_facesecure (1).pdf


In [16]:
#carregar e processar o conteúdo de um arquivo .pdf

from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "manual_facesecure.pdf"

loader = PyMuPDFLoader(file_path) #na chamada da função, o documento é lido e transformado em uma lista de objetos do tipo 'documents', cada um representando uma página do pdf com o texto extraído
doc = loader.load()
doc[0] #cada posição é uma página do documento

Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': '', 'creationdate': 'D:20250728031816', 'source': 'manual_facesecure.pdf', 'file_path': 'manual_facesecure.pdf', 'total_pages': 4, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': 'D:20250728031816', 'page': 0}, page_content='Manual de Atendimento e Uso - FaceSecure\n1. Introdução\nObjetivo do Manual\nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento completo do\nsistema de reconhecimento facial FaceSecure, abordando desde o cadastro, funcionalidades e\nserviços até procedimentos de segurança, suporte e resolução de problemas. O documento\ntambém serve como material de consulta para dúvidas frequentes.\nVisão geral da FaceSecure\nO FaceSecure é um sistema de reconhecimento facial que oferece autenticação segura, rápida e\ninteligente para diversos cenários: acesso a sistemas, c

In [17]:
#informações sobre o pdf
import pprint

pprint.pprint(doc[0].metadata)

{'author': '',
 'creationDate': 'D:20250728031816',
 'creationdate': 'D:20250728031816',
 'creator': '',
 'file_path': 'manual_facesecure.pdf',
 'format': 'PDF 1.3',
 'keywords': '',
 'modDate': '',
 'moddate': '',
 'page': 0,
 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/',
 'source': 'manual_facesecure.pdf',
 'subject': '',
 'title': '',
 'total_pages': 4,
 'trapped': ''}


In [18]:
#função para a extração dos textos

def extract_text_pdf(file_path):
  loader = PyMuPDFLoader(file_path)
  doc = loader.load()  #na variável doc há uma lista com cada uma das páginas
  content = "\n".join([page.page_content for page in doc]) #percorre cada pasta extraindo o texto e concatenando em um única variável
  return content

In [19]:
extract_text_pdf(file_path)

'Manual de Atendimento e Uso - FaceSecure\n1. Introdução\nObjetivo do Manual\nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento completo do\nsistema de reconhecimento facial FaceSecure, abordando desde o cadastro, funcionalidades e\nserviços até procedimentos de segurança, suporte e resolução de problemas. O documento\ntambém serve como material de consulta para dúvidas frequentes.\nVisão geral da FaceSecure\nO FaceSecure é um sistema de reconhecimento facial que oferece autenticação segura, rápida e\ninteligente para diversos cenários: acesso a sistemas, controle de entrada em ambientes físicos,\nverificação de identidade em aplicativos e muito mais. A solução é baseada em inteligência artificial\ne biometria facial, com foco em precisão, privacidade e facilidade de uso.\nPúblico-alvo\nA solução é voltada para empresas e usuários finais que buscam uma forma moderna e segura de\nautenticação, substituindo senhas e cartões por reconhecimento facial.\n2. 

In [23]:
#leitura de todos os arquivos .pdf contidos em uma pasta

docs_path = Path("/content") #pasta padrão do colab
pdf_files = [f for f in docs_path.glob("*.pdf")]

print(pdf_files)

[PosixPath('/content/manual_facesecure (1).pdf'), PosixPath('/content/manual_facesecure.pdf')]


In [29]:
loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files] #percorre cada um dos arquivos pdf e faz a extração por meio da função extract_file_pdf
len(pdf_files)

2

Divisão em pedaços de texto / Split

Neste estudo de caso, este passo é necessário pois documentos longos podem comprometer o desempenho da recuperação dos dados. Portanto, o texto será dividido em "pedaços" menores, pois o documento carregado é muito grande para ser passado como contexto pelo prompt.

Dependendo do conteúdo, poderia ser possível caber o conteúdo inteiro na janela de contexto. Por exemplo, o modelo GPT-4, a janela de contexto é de cerca de 8k tokens, o que equivale, aproximadamente, a 32k caracteres. Porém, mesmo assim, as LLMs terão dificuldade em localizar informações dentro de entradas tão longas.

Portanto, tal divisão do texto é considerada uma boa prática.

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_sppliter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)

#chunk_size: quantidade total de tokens em cada uma das partes;
#chunk_overlap: sobreposição de textos entre cada uma das partes
#chunk_overlap: quantos caracteres ou tokens do final do final de um chunk(ou parte) serão reutilizados no começo do próximo

chunks = []

for doc in loaded_documents:
  chunks.extend(text_sppliter.split_text(doc))

  print(f"Número de chunks criados:{len(chunks)}") #indica a quantidade de "pedaços" em que o contexto foi "quebrado"

Número de chunks criados:11
Número de chunks criados:22


Embeddings: é um modelo que organiza palavras por similaridade, agrupando termos relacionados no espaço vetorial.

Ao receber uma palavra nova, o modelo sabe onde colocá-la.

Com a compreensão de contexto através do Mecanismo de Atenção, o modelo diferencia os significados e posiciona corretamente os embeddings.

In [31]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = "sentence-transformers/all-mpnet-base-v2"
#embedding_model = "BAAI/bge-m3"

embeddings = HuggingFaceEmbeddings(model_name = embedding_model)

input_test = "Um teste apenas"

result = embeddings.embed_query(input_test)

In [32]:
print(result) #a frase "Um teste apenas" foi convertida para o seguinte embedding, que é valor um conjuntos de len(result) valores

[-0.026895038783550262, -0.011064727790653706, -0.04532000422477722, -0.0013972934102639556, 0.04243597015738487, -0.014201696030795574, 0.023354969918727875, 0.06011558324098587, 0.06152193248271942, 0.006502091884613037, 0.008159181103110313, -0.03056747280061245, 0.002060319297015667, 0.0129615543410182, 0.0042511713691055775, 0.0036631484981626272, -0.026755547150969505, 0.029737001284956932, -0.009770059026777744, -0.04650460556149483, -0.028108958154916763, 0.00016859530296642333, -0.02481103502213955, -0.01182831171900034, 0.0818282887339592, 0.0014993064105510712, 0.013264846988022327, -0.0624217726290226, -0.0012287232093513012, 0.02897718735039234, -0.02952898107469082, -0.02569606713950634, 0.0033771232701838017, -0.028868917375802994, 1.518517706244893e-06, -0.038824815303087234, -0.019842024892568588, -0.01765807531774044, -0.008112371899187565, -0.025130528956651688, 0.02574918232858181, 0.11367510259151459, -0.006133151240646839, -0.017986435443162918, -0.045411463826894

In [26]:
len(result)

768